In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
import random
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

#### Antes de hacer la Nueva Segmentación necesitamos unir la columna pk_cid con los 10 mil clientes con mejor probabilidad de compra con la tabla que se genera después de hacer el Encoding

In [ ]:
file_path = "C:/Users/Herre/Desktop/Entrega_TFM/df_encoding.csv"

df_original = pd.read_csv(file_path)

In [ ]:
file_path = "C:/Users/Herre/Desktop/Entrega_TFM/Prediciones.csv"

df_prediciones = pd.read_csv(file_path)

In [ ]:
# Filtrar el dataframe original para quedarte solo con los clientes presentes en df_combined
df_filtered = df_original[df_original['pk_cid'].isin(df_prediciones['pk_cid'])]

# Unir ambos dataframes en base a 'pk_cid'
df_final = df_filtered.merge(df_prediciones, on='pk_cid', how='left')

In [ ]:
# Mostrar las primeras filas del dataframe final
df_final.head(20)

Exportamos el dataframe para usarlo en la Nueva Segmentación

In [ ]:
# Guardar el DataFrame en un archivo CSV
df_final.to_csv('df_10kSegmentacion.csv', index=False)

#### Una vez hemos seleccionado a los 10 mil clientes con mejor probabilidad de compra necesitamos usarlos para hacer una nueva Segmentación de 4 clusters para crear una campaña de Marketing especializada para ellos.

In [ ]:
#Carga datos desde aquí
ruta_archivo = "C:/Users/Herre/Desktop/Entrega_TFM/df_10kSegmentacion.csv"

# Leer el archivo
df_clustering2 = pd.read_csv(ruta_archivo)

# Tabla Clustering_2

In [4]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering2)
# Perform K-Means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
df_clustering2['cluster'] = kmeans.fit_predict(X_scaled)

In [5]:
# Definir las columnas relevantes para el clustering
summary_columns = ["salary", "loan_amount", "debt_to_income_ratio", "account_balance", "number_of_products", "product_engagement_score",
"age", "segment", "active_customer", "region_code"]

# Inicializar el DataFrame ficha_df
ficha_df = pd.DataFrame()

# Generar los datos resumen para cada columna
for col in summary_columns:
    resumen_data = df_clustering2[["cluster", col]].groupby("cluster").describe().T[1:]
    ficha_df = pd.concat([ficha_df, resumen_data])

# Definir los componentes del multi-índice
out_index = ["Monetario"] *4  + ["Productos"] * 2 + ["Social"] * 4  # Ajusta esta lista según las columnas seleccionadas
inner_index = summary_columns
estadisticos = ["mean", "std", "min", "25%", "50%", "75%", "max"]

# Crear el nuevo multi-índice
new_multi_index = []
for oi, ii in zip(out_index, inner_index):
    for es in estadisticos:
        new_multi_index.append((oi, ii, es))

# Generar el multi-índice
def generate_multiindex(list_of_tuples, names):
    return pd.MultiIndex.from_tuples(list_of_tuples, names=names)

names = ["Grupo Indicadores", "Indicador", "Estadístico"]
index_ficha = generate_multiindex(new_multi_index, names)

# Establecer el multi-índice en ficha_df
ficha_df.set_index(index_ficha, inplace=True)

# Calcular el tamaño de cada cluster
tamaño_clusters = df_clustering2.groupby("cluster").size().to_frame().T
tamaño_clusters.set_index(generate_multiindex([("General", "Clúster", "Tamaño")], names), inplace=True)

# Concatenar los datos de tamaño con el DataFrame ficha_df
ficha_df = pd.concat([tamaño_clusters, ficha_df])

# Estilizar el DataFrame con un gradiente de fondo
styled_ficha_df = ficha_df.style.background_gradient(cmap='Blues', axis=1)

# Mostrar el DataFrame estilizado
styled_ficha_df

# Exportamos CSV

In [6]:
# Guardar resultados
df_clustering2.to_csv('df_clustering2.csv', index=False)